In [3]:
import pandas as pd
import numpy as np

### Use the [tabula](https://github.com/chezou/tabula-py) module to parse pdf, install locally if not present

In [4]:
try:
    import tabula
except:
    !pip install tabula-py --user

In [3]:
# !pip uninstall tabula-py --yes

Found existing installation: tabula-py 2.3.0
Uninstalling tabula-py-2.3.0:
  Successfully uninstalled tabula-py-2.3.0


### Select file address and load
The file contains most known synthesized perovskites. The chemical formula for single perovskite is ABO$_3$, where A and B sites are occupied by metals. The chemical formula for double perovskite is AA'BB'O$_3$, where A, A', B, B' sites are occupied by metals. The dataset used for this coding problem is taken from the supplementary materials of a [publication](https://www.pnas.org/doi/10.1073/pnas.1719179115).

In [5]:
fdir = r'.\pnas.1719179115.sd01.pdf'

In [6]:
from IPython.display import IFrame
IFrame(fdir, width=1000, height=600)

### Questions:
#### Q1. Finish the rest of the table parsing and processing
#### <font color='green'>A1. See below three equivalent solutions (A1.1$-$A1.3) with different code lengths. Although shorter answers are lightweight, they tend to be harder to debug.</font>

#### Q2. Find the most common A-site element in the listed perovskite materials
#### hint: use the [value_counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html) method
#### <font color='green'>A2. Ba or Barium</font>

In [ ]:
# Check how tabula.read_pdf works
df = tabula.read_pdf??

### A1.1 Solution (with pause at each step)

In [38]:
# Load pdf table into a dataframe
df = tabula.read_pdf(fdir, output_format='dataframe',
                     pages=list(range(1, 57)),
                     pandas_options={'index_col':list(range(0, 1)),
                                      'skipinitialspace':True})

In [39]:
# Select the first member of the return since the previous function call returns a list
df = df[0]

In [40]:
df.head()

,Source (ICSD ID or reference),A,B,B’,X,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
Compound,,,,,,,,,
NaN,NaN,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState
AgCoF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Co,2,Co,2,F,-1
AgCuF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Cu,2,Cu,2,F,-1
AgMgF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mg,2,Mg,2,F,-1
AgMnF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mn,2,Mn,2,F,-1


In [41]:
# Reset DataFrame index, which removes the multi-index table (i.e. two-level column names)
df.reset_index(inplace=True)
df.head()

,Compound,Source (ICSD ID or reference),A,B,B’,X,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState
1,AgCoF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Co,2,Co,2,F,-1
2,AgCuF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Cu,2,Cu,2,F,-1
3,AgMgF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mg,2,Mg,2,F,-1
4,AgMnF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mn,2,Mn,2,F,-1


In [42]:
# Extract coloumn names
old_names = list(df.columns)
old_names

['Compound',
 'Source (ICSD ID or reference)',
 'A',
 'B',
 'B’',
 'X',
 'Unnamed: 6',
 'Unnamed: 7',
 'Unnamed: 8',
 'Unnamed: 9']

In [43]:
# Construct a new dictionary for renaming columns
new_names = ['Compound', 'Source', 'A', 'A_oxi', 'B', 'B_oxi', 'B’', 'B’_oxi', 'X', 'X_oxi']
name_dict = dict(zip(old_names, new_names))
name_dict

{'Compound': 'Compound',
 'Source (ICSD ID or reference)': 'Source',
 'A': 'A',
 'B': 'A_oxi',
 'B’': 'B',
 'X': 'B_oxi',
 'Unnamed: 6': 'B’',
 'Unnamed: 7': 'B’_oxi',
 'Unnamed: 8': 'X',
 'Unnamed: 9': 'X_oxi'}

In [44]:
# Update the column names
df.rename(columns=name_dict, inplace=True)
df.head()

,Compound,Source,A,A_oxi,B,B_oxi,B’,B’_oxi,X,X_oxi
0,NaN,NaN,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState
1,AgCoF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Co,2,Co,2,F,-1
2,AgCuF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Cu,2,Cu,2,F,-1
3,AgMgF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mg,2,Mg,2,F,-1
4,AgMnF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mn,2,Mn,2,F,-1


In [45]:
# Drop the first row since it's not a material entry
df.drop(labels=0, axis=0, inplace=True)
df.head()

,Compound,Source,A,A_oxi,B,B_oxi,B’,B’_oxi,X,X_oxi
1,AgCoF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Co,2,Co,2,F,-1
2,AgCuF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Cu,2,Cu,2,F,-1
3,AgMgF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mg,2,Mg,2,F,-1
4,AgMnF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mn,2,Mn,2,F,-1
5,AgNbO3,55643,Ag,1,Nb,5,Nb,5,O,-2


In [46]:
# Count and rank the number of A-site elements. The output has two columns,
# the left one is the name of the A-site element, the right one is the occurence.
# The displayed results mean that the most common A-site element is Ba or Barium.

df['A'].value_counts()

Ba    289
Sr    245
Cs    208
Ca    129
Rb    118
La    102
Pb     59
Na     53
K      52
Nd     37
Pr     32
Sm     31
Eu     27
Gd     26
Tl     22
Li     20
Ag     17
Dy     15
Tb     15
Ho     14
Y      14
Cd     11
Er     11
Tm     10
Lu     10
Bi      9
Mn      7
Yb      7
Ce      7
Pu      4
U       3
Sc      3
Fe      2
Hg      2
In      2
Zn      1
li      1
Rv      1
Cr      1
Th      1
Mg      1
Cu      1
Name: A, dtype: int64

### A1.2 Integrated solution
Putting the solution in in a simpler form without intermediate outputs.

In [47]:
df2 = tabula.read_pdf(fdir, output_format='dataframe',
                     pages=list(range(1, 57)),
                     pandas_options={'index_col':list(range(0, 1)),
                                      'skipinitialspace':True})[0]

df2.reset_index(inplace=True)

In [48]:
old_names = list(df2.columns)
new_names = ['Compound', 'Source', 'A', 'A_oxi', 'B', 'B_oxi', 'B’', 'B’_oxi', 'X', 'X_oxi']
name_dict = dict(zip(old_names, new_names))

In [49]:
df2.rename(columns=name_dict, inplace=True)
df2.drop(labels=0, axis=0, inplace=True)

In [50]:
df['A'].value_counts()

Ba    289
Sr    245
Cs    208
Ca    129
Rb    118
La    102
Pb     59
Na     53
K      52
Nd     37
Pr     32
Sm     31
Eu     27
Gd     26
Tl     22
Li     20
Ag     17
Dy     15
Tb     15
Ho     14
Y      14
Cd     11
Er     11
Tm     10
Lu     10
Bi      9
Mn      7
Yb      7
Ce      7
Pu      4
U       3
Sc      3
Fe      2
Hg      2
In      2
Zn      1
li      1
Rv      1
Cr      1
Th      1
Mg      1
Cu      1
Name: A, dtype: int64

### A1.3 Leanest solution using method chaining
Requires to load in the table first, then apply the DataFrame transformation methods sequentially one by one, while setting the option ```inplace=False```

In [80]:
df3 = tabula.read_pdf(fdir, output_format='dataframe',
                     pages=list(range(1, 57)),
                     pandas_options={'index_col':list(range(0, 1)),
                                      'skipinitialspace':True})[0].reset_index(inplace=False)

In [81]:
df3.head()

,Compound,Source (ICSD ID or reference),A,B,B’,X,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,NaN,NaN,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState,Ion,Oxidation\rState
1,AgCoF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Co,2,Co,2,F,-1
2,AgCuF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Cu,2,Cu,2,F,-1
3,AgMgF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mg,2,Mg,2,F,-1
4,AgMnF3,"Acta Cryst B 64, 702-707 (2008)",Ag,1,Mn,2,Mn,2,F,-1


In [82]:
old_names = list(df3.columns)
new_names = ['Compound', 'Source', 'A', 'A_oxi', 'B', 'B_oxi', 'B’', 'B’_oxi', 'X', 'X_oxi']
name_dict = dict(zip(old_names, new_names))

In [83]:
df3.rename(columns=name_dict, inplace=False).drop(labels=0, axis=0, inplace=False)['A'].value_counts()

Ba    289
Sr    245
Cs    208
Ca    129
Rb    118
La    102
Pb     59
Na     53
K      52
Nd     37
Pr     32
Sm     31
Eu     27
Gd     26
Tl     22
Li     20
Ag     17
Dy     15
Tb     15
Ho     14
Y      14
Cd     11
Er     11
Tm     10
Lu     10
Bi      9
Mn      7
Yb      7
Ce      7
Pu      4
U       3
Sc      3
Fe      2
Hg      2
In      2
Zn      1
li      1
Rv      1
Cr      1
Th      1
Mg      1
Cu      1
Name: A, dtype: int64

Since ```inplace=False``` is also the default setting, this option can also be ignored to make the code even shorter.

In [84]:
df3.rename(columns=name_dict).drop(labels=0, axis=0)['A'].value_counts()

Ba    289
Sr    245
Cs    208
Ca    129
Rb    118
La    102
Pb     59
Na     53
K      52
Nd     37
Pr     32
Sm     31
Eu     27
Gd     26
Tl     22
Li     20
Ag     17
Dy     15
Tb     15
Ho     14
Y      14
Cd     11
Er     11
Tm     10
Lu     10
Bi      9
Mn      7
Yb      7
Ce      7
Pu      4
U       3
Sc      3
Fe      2
Hg      2
In      2
Zn      1
li      1
Rv      1
Cr      1
Th      1
Mg      1
Cu      1
Name: A, dtype: int64